In [1]:
# GET DATA
import pandas as pd
import numpy as np
from get_data import get_assets, get_positional_data

get_assets()
positions = get_positional_data()


This process will pip install Kaggle and download data through Kaggle API.

Please confirm that you've downloaded Kaggle JSON credentials into directory

Data Successfully Downloaded


In [2]:
positions.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,week
0,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN,1
1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN,1
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH,1
3,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN,1
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH,1


In [12]:
import form_pred
import importlib
importlib.reload(form_pred)
from form_pred import clean_positional

clean_pos = clean_positional(positions)

In [13]:
clean_pos.head()

,CBR0_x,DER0_x,DLL0_x,DLR0_x,DTL0_x,FBL0_x,FBR0_x,FSL0_x,HBL0_x,HBL1_x,...,WRL1_in,WRL2_in,WRL3_in,WRR0_in,WRR1_in,WRR2_in,WRR3_in,gameId,playId,offenseFormation
0,0.0,0.0,0.0,0.0,0.0,-5.12,0.00,0.0,0.0,0.0,...,True,False,False,False,False,False,False,2018090600,75,I_FORM
6,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,False,False,False,True,True,False,False,2018090600,146,SINGLEBACK
12,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,False,False,False,True,False,False,False,2018090600,168,SHOTGUN
18,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,False,False,False,True,True,True,False,2018090600,190,SHOTGUN
24,0.0,0.0,0.0,0.0,0.0,0.00,-4.59,0.0,0.0,0.0,...,False,False,False,False,False,False,False,2018090600,256,SHOTGUN


In [23]:
import ball_movement
import importlib
importlib.reload(ball_movement)
from ball_movement import ball_quadrants, make_quad_chart

quads = ball_quadrants(positions,4)

make_quad_chart(quads)

alt.Chart(...)

In [15]:
import def_clean
import importlib
importlib.reload(def_clean)
from def_clean import *

weeks =  range(1,15)

import time
start_time = time.time()

#week_data = pd.read_csv('Kaggle-Data-Files/week1.csv')
print('.......reading inputs.......')
play_data = pd.read_csv('Kaggle-Data-Files/plays.csv')
weeks_data = pd.read_csv('assets/full_position.csv')
print('')
print('.......starting processing.......')
output_df = pd.DataFrame()

for week in weeks:
    week_data = weeks_data[weeks_data['week']==week]
    print('...Week {} loaded...'.format(str(week)))
    filtered_df = filter_full_position_df(week_data, 11)
    print('...filtered...')
    transform_df = transform_directions(filtered_df, play_data)
    print('...transformed...')
    full_df, reduced_df = reduce_time(transform_df, 11)
    #print('...reduced...')
    simple_df = simple_closest_player(full_df, reduced_df)
    #print('...measured...')
    start_df = starting_pos(full_df)
    action_df = generate_action_type_df(simple_df)
    total_df =  pd.concat([action_df,start_df],axis=0)
    total_df['week'] = week
    print('...generated...')
    if output_df.empty:
        output_df = total_df.copy(deep=True)
    else:
        output_df = output_df.append(total_df)

    print('.....Week {} COMPLETE.....'.format(str(week)))
    print('')
    percent_complete = round(week / len(weeks)*100,2)
    print('   {}% COMPLETE   '.format(str(percent_complete)))
    print('')
    end_time = time.time()
    print("--- {} minutes elapsed ---".format(round((end_time - start_time)/60,1)))
    print('')

    print("the weeks complete: ", output_df.week.unique())

output_df = output_df.pivot(index=['gameId', 'playId'], columns='posId',values='value')
output_df.to_csv('assets/def_clean_output.csv')

..............................initializing
.......reading inputs.......

.......starting processing.......
...Week 1 loaded...
...filtered...
check for any offensive positions not mapped ['SS' 'FS' 'MLB' 'CB' 'LB' 'OLB' 'ILB' 'DL' 'DB' 'NT' 'S' 'DE']
...transformed...
...generated...
.....Week 1 COMPLETE.....

   7.14% COMPLETE   

--- 1.4 minutes elapsed' ---

...Week 2 loaded...
...filtered...
check for any offensive positions not mapped ['CB' 'SS' 'OLB' 'MLB' 'S' 'FS' 'ILB' 'DB' 'DL' 'LB' 'DE' 'P' 'LS']
...transformed...
...generated...
.....Week 2 COMPLETE.....

   14.29% COMPLETE   

--- 2.5 minutes elapsed' ---

...Week 3 loaded...
...filtered...
check for any offensive positions not mapped ['CB' 'ILB' 'OLB' 'S' 'SS' 'FS' 'MLB' 'LB' 'DB' 'DE' 'DL' 'K' 'P' 'LS']
...transformed...
...generated...
.....Week 3 COMPLETE.....

   21.43% COMPLETE   

--- 3.5 minutes elapsed' ---

...Week 4 loaded...
...filtered...
check for any offensive positions not mapped ['CB' 'FS' 'SS' 'ILB' 'OLB' 

In [17]:
d = pd.read_csv('assets/def_clean_output.csv')

0      1
1      1
2      1
3      1
4      1
      ..
910    1
911    1
912    1
913    1
914    1
Name: week, Length: 915, dtype: int64

In [19]:
d['week'].unique()


array([1])